## Deliverable 2. Create a Customer Travel Destinations Map.

In [18]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [19]:
# 1. Import the WeatherPy_database.csv file. 
#city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ancud,CL,-41.87,-73.82,61.00,58,75,10.29,broken clouds
1,1,Busselton,AU,-33.65,115.33,67.87,73,4,16.13,clear sky
2,2,Nauchnyy Gorodok,RU,53.42,83.52,10.40,78,75,13.42,light snow
3,3,Raudeberg,NO,61.99,5.14,41.00,75,75,2.24,broken clouds
4,4,Beyneu,KZ,45.32,55.20,30.65,89,6,10.42,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
minimum_temp = float(input("Please enter the Minimum temperature(in F) for your vacation."))
maximum_temp = float(input("Please enter the Maximum temperature(in F) for your vacation."))

Please enter the Minimum temperature(in F) for your vacation.50
Please enter the Maximum temperature(in F) for your vacation.80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_temp_data_df = city_data_df.loc[(city_data_df["Max Temp"] >= minimum_temp)
                                       & (city_data_df["Max Temp"] <= maximum_temp) ]
city_temp_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ancud,CL,-41.87,-73.82,61.00,58,75,10.29,broken clouds
1,1,Busselton,AU,-33.65,115.33,67.87,73,4,16.13,clear sky
5,5,Port Alfred,ZA,-33.59,26.89,69.01,82,37,7.94,scattered clouds
7,7,Illela,NG,13.73,5.30,77.38,18,34,5.70,scattered clouds
9,9,Ribeira Grande,PT,38.52,-28.70,60.80,72,40,21.92,scattered clouds
11,11,Hambantota,LK,6.12,81.12,77.79,84,46,13.38,scattered clouds
15,15,Key Biscayne,US,25.69,-80.16,78.80,61,75,23.04,broken clouds
21,21,Buta,CD,2.79,24.73,77.61,48,84,2.19,broken clouds
22,22,Mar Del Plata,AR,-38.00,-57.56,71.60,49,40,17.22,scattered clouds
23,23,Abha,SA,18.22,42.51,59.00,82,0,4.70,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
city_temp_data_df.count()

#city_temp_data_df.isnull().sum()

City_ID                296
City                   296
Country                291
Lat                    296
Lng                    296
Max Temp               296
Humidity               296
Cloudiness             296
Wind Speed             296
Current Description    296
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_temp_data_df.dropna()
clean_df.count()

City_ID                291
City                   291
Country                291
Lat                    291
Lng                    291
Max Temp               291
Humidity               291
Cloudiness             291
Wind Speed             291
Current Description    291
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ancud,CL,61.00,broken clouds,-41.87,-73.82,
1,Busselton,AU,67.87,clear sky,-33.65,115.33,
5,Port Alfred,ZA,69.01,scattered clouds,-33.59,26.89,
7,Illela,NG,77.38,scattered clouds,13.73,5.30,
9,Ribeira Grande,PT,60.80,scattered clouds,38.52,-28.70,
11,Hambantota,LK,77.79,scattered clouds,6.12,81.12,
15,Key Biscayne,US,78.80,broken clouds,25.69,-80.16,
21,Buta,CD,77.61,broken clouds,2.79,24.73,
22,Mar Del Plata,AR,71.60,scattered clouds,-38.00,-57.56,
23,Abha,SA,59.00,clear sky,18.22,42.51,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
        
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found")
        
print("All Hotel Data Retrieval Complete")

Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
All Hotel Data Retrieval Complete


In [14]:
# 7. Drop the rows where there is no Hotel Name.
#clean_hotel_df = hotel_df.dropna() #Does not work here
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df.count()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ancud,CL,61.00,broken clouds,-41.87,-73.82,Hotel Arena Gruesa
1,Busselton,AU,67.87,clear sky,-33.65,115.33,Observatory Guest House
5,Port Alfred,ZA,69.01,scattered clouds,-33.59,26.89,The Halyards Hotel
9,Ribeira Grande,PT,60.80,scattered clouds,38.52,-28.70,Quinta da Meia Eira
11,Hambantota,LK,77.79,scattered clouds,6.12,81.12,Bungalow 63
...,...,...,...,...,...,...,...
704,Saurimo,AO,66.96,light rain,-9.66,20.39,Hotel Kawissa Saurimo
707,Madisonville,US,57.99,moderate rain,37.33,-87.50,Days Inn by Wyndham Madisonville
713,Padang,ID,75.20,light rain,-0.95,100.35,Mercure Padang
714,Esmeralda,CU,78.80,scattered clouds,21.85,-78.12,Centro de Pioneros Exploradores


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))